# Books Recommender system using clustering | Collaborative based

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
!unzip "/content/archive(2).zip"

Archive:  /content/archive(2).zip
  inflating: books_data/books.csv    
  inflating: books_data/ratings.csv  
  inflating: books_data/users.csv    


In [12]:
books = pd.read_csv("books_data/books.csv", sep = ";",on_bad_lines="skip",encoding='latin-1')

<ipython-input-12-ed5b92e70e6c>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("books_data/books.csv", sep = ";",on_bad_lines="skip",encoding='latin-1')


In [13]:
books.sample(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
180117,0439249538,The Mitten Tree,Candace Christiansen,1997,"Scholastic, Inc.",http://images.amazon.com/images/P/0439249538.0...,http://images.amazon.com/images/P/0439249538.0...,http://images.amazon.com/images/P/0439249538.0...
33706,0140707085,Henry V (New Penguin Shakespeare),William Shakespeare,1981,Penguin Books,http://images.amazon.com/images/P/0140707085.0...,http://images.amazon.com/images/P/0140707085.0...,http://images.amazon.com/images/P/0140707085.0...
43706,0553263609,Nine Stories,J.D. Salinger,1984,Bantam Books,http://images.amazon.com/images/P/0553263609.0...,http://images.amazon.com/images/P/0553263609.0...,http://images.amazon.com/images/P/0553263609.0...
28239,069452008X,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1998,HarperAudio,http://images.amazon.com/images/P/069452008X.0...,http://images.amazon.com/images/P/069452008X.0...,http://images.amazon.com/images/P/069452008X.0...
41727,0027476626,Boris the Boring Boar,Ellen Jackson,1992,Simon &amp; Schuster,http://images.amazon.com/images/P/0027476626.0...,http://images.amazon.com/images/P/0027476626.0...,http://images.amazon.com/images/P/0027476626.0...


In [14]:
books = books[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','Image-URL-M']]
books.shape

(271360, 6)

In [15]:
books.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2
Image-URL-M,0


In [16]:
books = books.dropna()
books.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,0
Year-Of-Publication,0
Publisher,0
Image-URL-M,0


In [17]:
users = pd.read_csv("books_data/users.csv", sep = ";",on_bad_lines="skip",encoding='latin-1')

In [18]:
users.sample(5)

,User-ID,Location,Age
172506,172507,"fairfax, virginia, usa",33.0
74236,74237,"copenhagen, n/a, denmark",25.0
163110,163111,"atlanta, georgia, usa",NaN
248876,248877,"little rock, arkansas, usa",57.0
58782,58783,"santa cruz, california, usa",NaN


In [19]:
users.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


In [20]:
ratings = pd.read_csv("books_data/ratings.csv", sep = ";",on_bad_lines="skip",encoding='latin-1')

In [21]:
ratings.sample(5)

,User-ID,ISBN,Book-Rating
113995,26516,0671027123,9
1049503,250962,0811808173,0
846376,204864,0380709457,0
718914,174072,0060976845,7
1065045,254465,0505522888,0


In [22]:
ratings.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


In [23]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271356, 6)
(278858, 3)
(1149780, 3)


In [24]:
ratings.rename(columns={
    "User-ID" : "user_id",
    "Book-Rating" : "rating"
},inplace=True)

ratings.sample(5)

,user_id,ISBN,rating
151505,34501,0312963009,0
329395,78783,0312981961,0
423601,101209,0307987698,8
549195,131675,9024215234,0
1048463,250764,0394728564,0


In [25]:
ratings['user_id'].value_counts()

,count
user_id,
11676,13602
198711,7550
153662,6109
98391,5891
35859,5850
...,...
116180,1
116166,1
116154,1


In [26]:
x = ratings['user_id'].value_counts()>200
x

,count
user_id,
11676,True
198711,True
153662,True
98391,True
35859,True
...,...
116180,False
116166,False
116154,False


In [27]:
y = x[x].index

In [28]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [29]:
ratings = ratings[ratings['user_id'].isin(y)]

In [30]:
ratings.head(5)

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [31]:
ratings.shape

(526356, 3)

In [32]:
ratings_with_books = ratings.merge(books, on = "ISBN")
ratings_with_books.rename(columns={'Book-Title':'title'},inplace=True)

In [33]:
ratings_with_books.head(2)

,user_id,ISBN,rating,title,Book-Author,Year-Of-Publication,Publisher,Image-URL-M
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [34]:
ratings_with_books.shape

(487668, 8)

In [35]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [36]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [37]:
num_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [38]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [39]:
final_rating = ratings_with_books.merge(num_rating,on='title')

In [40]:
final_rating.head(2)

,user_id,ISBN,rating,title,Book-Author,Year-Of-Publication,Publisher,Image-URL-M,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [41]:
final_rating.shape

(487668, 9)

In [42]:
final_rating = final_rating[final_rating['num_of_rating']>=50]

In [43]:
final_rating.head()

,user_id,ISBN,rating,title,Book-Author,Year-Of-Publication,Publisher,Image-URL-M,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [44]:
final_rating.shape

(61853, 9)

In [45]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [46]:
final_rating.shape

(59850, 9)

In [47]:
book_pivot = final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [48]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [49]:
book_pivot.shape

(742, 888)

In [50]:
book_pivot.fillna(0,inplace = True)

In [51]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
from scipy.sparse import csr_matrix


In [53]:
book_sparse = csr_matrix(book_pivot)

In [54]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

In [55]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [56]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [57]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)

In [58]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [59]:
suggestion

array([[237, 240, 238, 241, 184, 536]])

In [64]:
for i in range(len(suggestion)):
  print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [61]:
books_name = book_pivot.index

In [63]:
# ! mkdir 'artifacts'
import pickle
pickle.dump(model, open('artifacts/model.pkl','wb'))
pickle.dump(books_name, open('artifacts/book_name.pkl','wb'))
pickle.dump(final_rating, open('artifacts/final_ratings.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_piot.pkl','wb'))

In [69]:
def recommend_book(book_name):
  book_id = np.where(book_pivot.index==book_name)[0][0]
  distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
  for i in range(len(suggestion)):
    books = book_pivot.index[suggestion[i]]
    for j in books:
      print(j)


In [72]:
recommend_book('Harry Potter and the Chamber of Secrets (Book 2)')

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall
